In [1]:
import numpy as np
from skimage import io
import skimage.morphology as skmp
import matplotlib.pyplot as plt
from random import randint
from tqdm import tqdm

In [3]:
# hyperparameters and other global variables
input_path = './sample_data/'
output_path = '/Users/advithchegu/Desktop/Rutgers/Spring 22/Computer Vision/hw/Assignment2/out/'
out_height = 200
out_width = 200
window_size = 9
sample_img = 'test_im1.bmp'
ErrThreshold = 0.1
MaxErrThreshold = 0.3
Sigma = window_size / 6.4
seed_size = 3
half_window = int((window_size - 1) / 2)

In [4]:
# we need to first read the sample image and convert it to a binary representation
SampleImage = io.imread(input_path + sample_img)

io.imshow(SampleImage)
plt.title('Original Image')
io.show()

# shrink to range between 0 and 1
SampleImage = SampleImage / 255.0

# find the first non-zero value
# np.savetxt("image.csv", SampleImage, delimiter=",")

FileNotFoundError: ignored

In [4]:
'''
Let SampleImage contain the image we are sampling from and let Image be the mostly empty 
image that we want to fill in (if synthesizing from scratch, it should contain a 3-by-3 
seed in the center randomly taken from SampleImage, for constrained synthesis it should 
contain all the known pixels). 
'''

# actual_x1 = 20
# actual_x2 = 298
# actual_y1 = 20
# actual_y2 = 211

# plus one because the last index doesn't count
Image = SampleImage

# map to mark the pixels that we have synthesized already
MarkerMap = np.ceil(Image)

In [5]:
'''
Before we go on let's find all the possible patches in the original image that we can 
convolve with to find the most similar patches in the generation process
'''

possible_conv = []

for i in range(out_height - window_size):
    for j in range(out_width - window_size):
        # append all of the possible patches we can convolve against to measure similarity
        possible_conv.append(np.reshape(SampleImage[i:i+window_size, j:j+window_size], (window_size, window_size)))

In [6]:
'''
This part generates a two-dimensional Gaussian in a window of given a size centered in 
the center and with a given standard deviation (in pixels)
'''

# this code is taken straight from MATLAB documentation 'fspecial' gaussian
x, y = np.mgrid[-window_size//2 + 1:window_size//2 + 1, -window_size//2 + 1:window_size//2 + 1]
g = np.exp(-((x**2 + y**2)/(2.0*Sigma**2)))
GaussMask = g/g.sum()

# also let's pad the images with a half window because it's just easier
Image_Pad = np.lib.pad(Image, half_window, mode='constant', constant_values=0)
Marker_Pad = np.lib.pad(MarkerMap, half_window, mode='constant', constant_values=0)

In [7]:
'''
This part is the loop, we repeat until all pixels are filled
'''

pbar = tqdm(total=np.count_nonzero(MarkerMap==0.0))

while 0 in MarkerMap:
    # indicator var
    progress = False

    # returns a list of all unfilled pixels that have filled pixels as their neighbors 
    # (the image is subtracted from its morphological dilation)
    PixelList = np.transpose(np.nonzero(skmp.binary_dilation(MarkerMap) - MarkerMap))

    for pixel in PixelList:
        # gets the window around the pixel we use a half window so that the pixel is at the center
        Template = Image_Pad[pixel[0]: pixel[0] + 2*half_window + 1, pixel[1]: pixel[1] + 2*half_window + 1]
        ValidMask = Marker_Pad[pixel[0]: pixel[0] + 2*half_window + 1, pixel[1]: pixel[1] + 2*half_window + 1]

        # reshape so we avoid the divide by zero errs
        Template = np.reshape(Template, (window_size, window_size))
        ValidMask = np.reshape(ValidMask, (window_size, window_size))
        GaussMask = np.reshape(GaussMask, (window_size, window_size))

        # print(ValidMask)

        # now we have to find the best matches between the possible convolutions and template
        TotWeight = np.sum(np.multiply(Template, ValidMask))
        ssd_arr = []
        min_ssd = float('inf')

        # traverse the possible convolutions and calculate distances
        for c in possible_conv:
            dist = np.subtract(Template, c)**2
            ssd_arr.append((np.sum(np.divide((dist * ValidMask * GaussMask), TotWeight)), c))
        
        # return all of the possible convolutions that are in an acceptable error thresh
        min_err = min(ssd_arr, key = lambda t: t[0])
        BestMatches = [(err, c[half_window][half_window]) for err, c in ssd_arr if err <= min_err[0]*(1+ErrThreshold)]
        BestMatch = randint(0, len(BestMatches)-1)

        # check if the error is less than the acceptable amount
        if BestMatches[BestMatch][0] < MaxErrThreshold:
            Image_Pad[half_window+pixel[0]][half_window+pixel[1]] = BestMatches[BestMatch][1]
            Image[pixel[0]][pixel[1]] = BestMatches[BestMatch][1]
            Marker_Pad[half_window+pixel[0]][half_window+pixel[1]] = 1
            MarkerMap[pixel[0]][pixel[1]] = 1

            progress = True
            pbar.update(1)
        
        # increase threshold if it's not working
        if not progress:
            MaxErrThreshold *= 1.1

 16%|█▋        | 3792/23232 [34:20<2:56:10,  1.84it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in true_divide


ValueError: ignored

In [1]:
print(total=np.count_nonzero(MarkerMap==0.0))

NameError: ignored